This is a script to get 1 year median image for the 'envelope' of the Charlotte Metro Areas Counties

Simply change dates in search and image printing functions

It takes a while to run (processing a year's worth of images for a rather large study area)

Key Points:

To be plotted a (multi)polygon must be in form: geopandas.geodataframe.GeoDataFrame

To run 'intersects' it must be a str representing a geojson geometry

To display on leaflet map must be a dict representing a geojson geometry

To use as mask for xarray 'clip' it must be a list (of coordinates)

In [1]:
import geopandas as gpd
import geojson
import json
import pandas as pd
import shapely
from shapely.geometry import mapping
from shapely.geometry import box
import folium


In [2]:

# import all shapefile of all counties in North Carolina and South Carolina
nc_counties = gpd.read_file("NC_counties/North_Carolina_State_and_County_Boundary_Polygons.shp")
sc_counties = gpd.read_file("SC_counties/south-carolina-county-boundaries.shp")

# Prep each dataframe to be merged
nc_counties.rename(columns={'County': 'NAME'}, inplace=True)
sc_counties.rename(columns={'CTYNAME': 'NAME'}, inplace=True)
nc_counties['NAME'] = nc_counties['NAME'].str.upper()

# Keep only 'NAME' and 'geometry' columns
nc_counties = nc_counties[['NAME', 'geometry']]
sc_counties = sc_counties[['NAME', 'geometry']]

# Assign 'STATE' attribute
nc_counties['STATE'] = 'North Carolina'
sc_counties['STATE'] = 'South Carolina'

# Convert both to 4326
nc_counties = nc_counties.to_crs(epsg=4326)
sc_counties = sc_counties.to_crs(epsg=4326)

# Now merge the two DataFrames
counties_all = pd.concat([nc_counties, sc_counties], ignore_index=True)

counties_all

,NAME,geometry,STATE
0,CAMDEN,"POLYGON ((-75.90629 36.08587, -75.90663 36.085...",North Carolina
1,GATES,"POLYGON ((-76.69658 36.29618, -76.69673 36.296...",North Carolina
2,IREDELL,"POLYGON ((-80.94811 35.49116, -80.94835 35.491...",North Carolina
3,WILKES,"POLYGON ((-81.30256 36.00490, -81.30285 36.004...",North Carolina
4,UNION,"POLYGON ((-80.55036 35.20840, -80.55007 35.208...",North Carolina
...,...,...,...
141,HORRY,"POLYGON ((-79.07716 34.29608, -79.07734 34.296...",South Carolina
142,HAMPTON,"POLYGON ((-81.08202 33.02657, -81.08171 33.026...",South Carolina
143,SUMTER,"POLYGON ((-80.47952 34.11239, -80.47525 34.110...",South Carolina
144,AIKEN,"POLYGON ((-81.57339 33.88026, -81.57326 33.880...",South Carolina


In [3]:
# We are using the Metropolitan Statistical Area (MSA) boundaries, not the Combined Statistical Area (CSA) boundary
# See https://en.wikipedia.org/wiki/Charlotte_metropolitan_area
# To run analyis with for the CSA read in 'Charlotte_metropolitan_area_2.csv' and concatenate
metro = pd.read_csv('Charlotte_metropolitan_area_1.csv')
metro

,County,2022 Estimate,2020 Census,Change,Area,Density
0,Mecklenburg County,"1,145,392","1,115,482",+2.68%,"523.84 sq mi (1,356.7 km2)","2,187/sq mi (844/km2)"
1,York County,"294,248","282,090",+4.31%,"680.60 sq mi (1,762.7 km2)",432/sq mi (167/km2)
2,Union County,"249,070","238,267",+4.53%,"631.52 sq mi (1,635.6 km2)",394/sq mi (152/km2)
3,Cabarrus County,"235,797","225,804",+4.43%,361.75 sq mi (936.9 km2),652/sq mi (252/km2)
4,Gaston County,"234,215","227,943",+2.75%,356.03 sq mi (922.1 km2),658/sq mi (254/km2)
5,Iredell County,"195,897","186,693",+4.93%,"573.83 sq mi (1,486.2 km2)",341/sq mi (132/km2)
6,Rowan County,"149,645","146,875",+1.89%,"511.37 sq mi (1,324.4 km2)",293/sq mi (113/km2)
7,Lancaster County,"104,577","96,016",+8.92%,"549.16 sq mi (1,422.3 km2)",190/sq mi (74/km2)
8,Lincoln County,"93,095","86,810",+7.24%,297.94 sq mi (771.7 km2),312/sq mi (121/km2)
9,Chester County,"31,931","32,294",−1.12%,"580.66 sq mi (1,503.9 km2)",55/sq mi (21/km2)


In [4]:
# Remove row showing total
metro1 = metro.drop(metro.index[-1:])
# Convert to uppercase to match format of County list
metro2 = metro1['County'].str.upper()
# Remove "COUNTY" and use .str.strip() to remove any extra spaces
metro3 = metro2.str.replace('COUNTY', '').str.strip()
metro3

0     MECKLENBURG
1            YORK
2           UNION
3        CABARRUS
4          GASTON
5         IREDELL
6           ROWAN
7       LANCASTER
8         LINCOLN
9         CHESTER
10          ANSON
Name: County, dtype: object

In [5]:
# Check how the datatype evolves, if wanted
# Using .str.upper() turns in into a 'series'
print(type(metro))
print(type(metro1))
print(type(metro2))
print(type(metro3))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [6]:
# Alternate way to process the data without changing it to a series
# but for our purposes here, .str acessors are sufficient
'''
metro1['County'] = metro1['County'].apply(lambda x: x.upper())
metro1
'''

"\nmetro1['County'] = metro1['County'].apply(lambda x: x.upper())\nmetro1\n"

In [7]:
# Filter the counties in nc_counties based on the processed county list
study_area = counties_all[counties_all['NAME'].isin(metro3)]
study_area = study_area.sort_values(by='NAME')
study_area = study_area.reset_index(drop=True)
# The len should be 10 (11 rows in the table)
print (len(study_area))
study_area

12


,NAME,geometry,STATE
0,ANSON,"POLYGON ((-80.27683 35.19572, -80.27662 35.195...",North Carolina
1,CABARRUS,"POLYGON ((-80.55036 35.20840, -80.55064 35.208...",North Carolina
2,CHESTER,"POLYGON ((-81.47866 34.82153, -81.42472 34.821...",South Carolina
3,GASTON,"POLYGON ((-80.95486 35.40008, -80.95482 35.399...",North Carolina
4,IREDELL,"POLYGON ((-80.94811 35.49116, -80.94835 35.491...",North Carolina
5,LANCASTER,"POLYGON ((-80.90645 35.07681, -80.90440 35.074...",South Carolina
6,LINCOLN,"POLYGON ((-80.96003 35.54702, -80.96000 35.546...",North Carolina
7,MECKLENBURG,"POLYGON ((-81.03390 35.14869, -81.03157 35.148...",North Carolina
8,ROWAN,"POLYGON ((-80.18256 35.50415, -80.18290 35.504...",North Carolina
9,UNION,"POLYGON ((-80.55036 35.20840, -80.55007 35.208...",North Carolina


In [8]:
# Oops, there are two Union Counties!
# Referencing the Wikipedia, we want to keep the one in North Carolina

# Filter out the row with NAME='UNION' and STATE='South Carolina'
study_area = study_area[~((study_area['NAME'] == 'UNION') & (study_area['STATE'] == 'South Carolina'))]
study_area = study_area.reset_index(drop=True)
study_area = study_area.set_crs(epsg=4326)

print(len(study_area))
print(study_area.crs)
study_area

11
EPSG:4326


,NAME,geometry,STATE
0,ANSON,"POLYGON ((-80.27683 35.19572, -80.27662 35.195...",North Carolina
1,CABARRUS,"POLYGON ((-80.55036 35.20840, -80.55064 35.208...",North Carolina
2,CHESTER,"POLYGON ((-81.47866 34.82153, -81.42472 34.821...",South Carolina
3,GASTON,"POLYGON ((-80.95486 35.40008, -80.95482 35.399...",North Carolina
4,IREDELL,"POLYGON ((-80.94811 35.49116, -80.94835 35.491...",North Carolina
5,LANCASTER,"POLYGON ((-80.90645 35.07681, -80.90440 35.074...",South Carolina
6,LINCOLN,"POLYGON ((-80.96003 35.54702, -80.96000 35.546...",North Carolina
7,MECKLENBURG,"POLYGON ((-81.03390 35.14869, -81.03157 35.148...",North Carolina
8,ROWAN,"POLYGON ((-80.18256 35.50415, -80.18290 35.504...",North Carolina
9,UNION,"POLYGON ((-80.55036 35.20840, -80.55007 35.208...",North Carolina


In [9]:
SA_combined = study_area.unary_union
combined_df = gpd.GeoDataFrame(geometry=[SA_combined])

In [10]:
type(SA_combined)

shapely.geometry.polygon.Polygon

In [11]:
# Save as shapefiles
# combined_df.to_file("metroArea.shp")
# study_area.to_file("Counties.shp")

In [12]:
# In order to visualize, create json version of study area
combined_df = combined_df.set_crs(epsg=4326)
combined_json = combined_df.to_json()

geometry_part = geojson.loads(combined_json)["features"][0]["geometry"]

type(geometry_part)

geojson.geometry.Polygon

In [16]:
# Display the study area and Landsat vs Sentinel swaths
# This takes a while to run and is not really necessary
m = folium.Map(location=[35.225776, -80.841026],
               zoom_start=8, 
               #height=800
              )



# Iterate over the rows of the GeoDataFrame
for index, row in study_area.iterrows():
    # Convert the geometry to a GeoJSON-like dictionary
    feature_geometry = mapping(row.geometry)
    
    # Add a GeoJSON-like dictionary as a GeoJSON layer
    folium.GeoJson(
        feature_geometry,
        name=f"{index}: {row['NAME']} COUNTY",
        style_function=lambda feature: {'color': 'blue', 'weight': 2, 'fillColor': 'blue'},
        tooltip=f"{index}: {row['NAME']} COUNTY"
    ).add_to(m)

folium.GeoJson(
    geometry_part,
    name=f"Study Area",
    style_function=lambda feature: {'color': 'red', 'weight': 3, 'fillColor': 'clear'},
    tooltip=f"Study Area"
).add_to(m)

    
# Add Layer Control
folium.LayerControl().add_to(m)

m